In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import os

from matplotlib import pyplot as plt
from dotenv import load_dotenv

load_dotenv()
plt.style.use('Solarize_Light2')

# Setting default DPI, pulling it from dotenv if it exists, setting it on 100 if not

pc_dpi = os.getenv('DPI')

if pc_dpi is None:
    pc_dpi = 100

pc_dpi = int(pc_dpi)


In [ ]:
# File

sea_ds = "./data/seattle_data.csv"

df_seattle = pd.read_csv(sea_ds)


In [ ]:
df_seattle.shape

# <u>Analyse exploratoire :</u>

### <u>1. Age des batiments </u>
- <u>1.a : Représentation des batiments en fonction de leurs dates de construction. </u>
- <u>1.b : Visualisation des statistiques principales de la variable : Energie par metre carré </u>
- <u>1.c : Paire 1 : Date de construction et utilisation en energie par metre carré. </u>

<hr>

### <u> 2. Etude de la note Energy Star</u>
- <u> 2.a : Repartition de la note Energy Star au sein des données. </u>
- <u> 2.b : Paire 2 : Note Energy Star en fonction de l'energie par metre carré. </u>
- <u> 2.c : Paire 3 : Energy Star en fonction de l'année de construction. </u>
<hr>


### <u> 3 : Représentation Property Type/Utilisation Energie </u>

### Cette analyse permettra de :
 - Visualiser "l'age" du paysage de Seattle
 - Contextualiser la mesure de la consommation d'énergie au metre carré, et sa possible relation avec l'age des batiments
 - Avoir une meilleure comprehension de la variable Energy Star et de la présenter face à l'age des batiments et de leur consommation en energie


## 1. Age des batiments

### 1.a : Représentation des batiments en fonction de leurs dates de construction.

In [ ]:
data_age = list(df_seattle["YearBuilt"].astype(int))
oldest = min(data_age)
youngest = max(data_age)

print(f"Le plus ancien batiment date de {oldest}, le plus récent (donnees de 2016) date de {youngest}")


In [ ]:
decades = np.arange(1900, 2020, 10)

building_per_decade = dict.fromkeys(decades)

for decade in building_per_decade.keys():
    built = len([year for year in data_age if (year > decade and year < decade + 10)])
    building_per_decade[decade] = built

used_cmap = plt.get_cmap("Dark2")

fig, (ax1, ax2) = plt.subplots(
    ncols=2,
    nrows=1,
    figsize=(22, 10),
    gridspec_kw={"width_ratios": [1, 4]},
    dpi=pc_dpi
    )

ax1.boxplot(data_age, showmeans=True, widths=0.25)

ax2.bar(building_per_decade.keys(), height=building_per_decade.values(), width=8, color=used_cmap.colors)
# ax2 lineplot

###
# Titles & Labels
ax1.set_ylabel("Annee de construction")
ax2.set_ylabel("Nombre de batiments construits")
ax2.set_xlabel("Annee de construction")
fig.suptitle("Representation des batiments en fonction de leur annee de construction")
fig.tight_layout()
#
###

plt.show()


#### Observations :

- Le paysage urbanin de Seattle date, majoritairement (q1, q3) de 1939 - 1990 (moyenne en 1965)
- On remarque une logique économique : pic de construction dans les années 20 (Roaring 20s) suivi par un creux dans les années 30 (Grande Depression)
- De nombreux projets date de du debut du millénaire, la decenie 2010+ n'est pas étudiable : les données s'arretent en 2015

### 1.b : Visualisation des statistiques principales de la variable : Energie par metre carré

In [ ]:
fig, (ax1, ax2) = plt.subplots(
    ncols=2,
    nrows=1,
    figsize=(22, 12),
    dpi=pc_dpi,
    )

source_eui_data = list(df_seattle["SourceEUI(kWh/m2)"].values.astype(float))

sns.boxplot(data=source_eui_data, width=0.6, ax=ax1, color="navy")
ax1.set_ylim(0, 1400)  # High number of outliers, cf describe
sns.boxplot(data=source_eui_data, width=0.6, ax=ax2, color="red")
# fig.tight_layout()

###
# Titles/Lables
ax1.set_ylabel("Consommation d'energie (kWh/m2)")
fig.suptitle("Visualisation de la repartition de l'EUI parmi les données (energy use intensity)\
 gauche : zoom sur 0-1400kW/m2")
#
###

plt.show()

print(df_seattle["SourceEUI(kWh/m2)"].astype(float).describe())


#### Analyse :

- 1 : Les outliers sont extrèmement nombreux, uniquement dans les utilisations "hautes"
- 2 : La majorité des batiments ont une EUI entre 247 et 592 kWh/m2, 519 en moyenne.
- 3 : L'élimination de batiments via la methode interquartile peut être viable, la reduction de dimension via ACP peut également être une piste


### 1.c : Paire 1 : Date de construction et utilisation en energie par metre carré

In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 12),
    dpi=pc_dpi,
)

ax1 = sns.barplot(
    x=df_seattle["YearBuilt"].astype(int).values,
    y=df_seattle["SourceEUI(kWh/m2)"].values.astype(float),
    ci=5
)

###
# Titles/Lables
ax1.set_xticklabels([])
ax1.set_ylabel("SourceEUI(kWh/m2)")
ax1.set_xlabel("Année de construction (1900 - 2015)")
fig.suptitle("Visualisation de 'energy use intensity' moyenne en fonction de l'année de construction des batiments")

#
###
fig.tight_layout()
plt.show()


#### Observation : 
- L'age des batiments ne semble pas avoir d'impact sur leur intensité d'utilisation énergétique. On peut emettre l'hypothèse que les batiments les plus anciens ont été modernisés pour avoir une consommation energetique plus responsable.

## 2. Etude de la note Energy Star

### 2.a : Repartition de la note Energy Star au sein des données

In [ ]:
fig, (ax1, ax2) = plt.subplots(
    ncols=2,
    nrows=1,
    figsize=(22, 12),
    gridspec_kw={"width_ratios": [1, 4]},
    dpi=pc_dpi,
)

rated = df_seattle[df_seattle["ENERGYSTARScore"].notna()]
scores = list(rated["ENERGYSTARScore"].astype(int).values)
score_range = range(0, 101, 1)
scores_dict = dict.fromkeys(score_range)

for score in scores_dict.keys():
    scores_dict[score] = scores.count(score)

ax1.boxplot(scores, showmeans=True, widths=0.25)

ax2.bar(
    x=list(scores_dict.keys()),
    height=list(scores_dict.values()),
    color=used_cmap.colors,
    width=1
    )

####
## Titles/Lables
ax1.set_ylabel("Nombre de batiments")
ax1.set_xlabel("Note Energy Star (0-100)")
fig.suptitle("Répartition de la note Energy Star au sein du dataset (2016)")
##
####

fig.tight_layout()

plt.show()

rated["ENERGYSTARScore"].describe()

#### Observations :
- De nombreux batiments (20/1100, ie +|- 2%) ont la note la plus basse (0)
- Les notes sont globalement hautes (48-89) - Les batiments ont le droit d'afficher le label "Energy Star Certified" si leur note est superieure ou égale a 75 -> Mediane (sources), motivation pour chercher a atteindre ce score.
- Variance haute (28). Comme énoncé plus haut, on utilisera sans doute la methode interquartile pour indentifier et se debarasser des valeurs aberrantes.

### 2.b : Paire 2 : Note Energy Star en fonction de l'energie par metre carré.

In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 10),
    dpi=pc_dpi,
)

ax1 = sns.barplot(
    x=rated["ENERGYSTARScore"].values.astype(int),
    y=rated["SourceEUI(kWh/m2)"].values.astype(float),
    ci=10
)

x_line = np.arange(0, 101, 5)

y_line = []

for x in x_line:
    ratings_x = rated[rated["ENERGYSTARScore"] == x]
    mean_x = ratings_x["SourceEUI(kWh/m2)"].mean(numeric_only=None)
    if pd.isna(mean_x):
        mean_x = rated["ENERGYSTARScore"].quantile(0.25).mean()
    y_line.append(mean_x)

ax1.plot(x_line, y_line)

###
# Titles/Lables
ax1.set_xticks(np.arange(-1, 101, 5))
ax1.set_ylabel("SourceEUI(kWh/m2)")
ax1.set_xlabel("Energy Star Score (0-100)")
fig.suptitle("Visualisation de 'l'energy use intensity' en fonction de la note E* des batiments")
#
###

fig.tight_layout()
plt.show()


#### Observation 
- On peut observer ici un lien clair entre le Score Energy Star (E*) et l'intensité d'utilisation énergétique (EUI). Le lien est logique dans ce cas : le site E* explique que, entre autres, l'utilisation d'énergie est un facteur.
- Le site internet d'E* fourni également une motivation pour l'obtention de cette note : un batiment est "Energy Star Certified" lorsque sa note est supérieure ou égale à 75 (ce qui peut expliquer le pic autour de cette note : les developpeurs immobiliers ont interèt à obtenir cette certification)

### 2.c Paire 3 : Rapport entre E* et année de construction :

In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 12),
    dpi=pc_dpi,
)

years_list = list(df_seattle[df_seattle["YearBuilt"].notna()]["YearBuilt"].values)
year_keys = []
for year in years_list:
    if year not in year_keys:
        year_keys.append(year)

year_keys.sort()

year_dict = dict.fromkeys(year_keys)

for key in year_dict.keys():
    temp = df_seattle[(df_seattle["YearBuilt"] == key) & (df_seattle["ENERGYSTARScore"].notna())]
    scores = [int(i) for i in temp["ENERGYSTARScore"]]
    try:
        mean = sum(scores) / len(scores)
        year_dict[key] = mean
    except ZeroDivisionError:
        year_dict[key] = 0

ax1.bar(
    x=list(year_dict.keys()),
    height=list(year_dict.values()),
    color=used_cmap.colors,
    width=1,
    )

###
# Titles/Lables
ax1.set_yticks(np.arange(0, 100, 5))
ax1.set_ylabel("Note Energie Star Moyenne")
ax1.set_xlabel("Année de construction (1900, 2015)")
fig.suptitle("Représentation de la moyenne E* des batiments en fonction de leur année de construction")
#
###

fig.tight_layout()
plt.show()


#### Observation :
- L'année de construction ne semble pas de lien clair avec la note E* - Cela peut venir, comme ecrit plus haut, de travaux de modernisation par exemple.

## 3 : Représentation Property Type/Utilisation Energie

<i>On cherche s'il existe des correllations entre les principaux types de batiments et leut note E*</i>

In [ ]:
uniques = df_seattle["PrimaryPropertyType"].unique()
unique_ptype_dict = dict.fromkeys(uniques)
for unique in unique_ptype_dict.keys():
    unique_ptype_dict[unique] = len(df_seattle[df_seattle["PrimaryPropertyType"] == unique])

unique_ptype_dict


#### Observations :
- SPS -> Seattle Public School
- K12 -> Shool -> from kindergarten to 12th grade
- Typos, doubles espaces etc.
- Pré groupement via généralisation
- La generalisation permettra de creer des categories plus inclusives tout en gardant les propriétes principales des batiments
- Necessité de clarification : Batiments types cités universitaires & campus comptent comme residentiels ?

In [ ]:
correction_dict = {
    "Self-Storage Facility\n": "Self-Storage Facility",  # Possibly warehouse
    "Small- and Mid-Sized Office": "Small & Mid-Sized Office",
    "Restaurant\n": "Restaurant",
    "Distribution Center\n": "Supermarket & Grocery store",
    "Distribution Center": "Supermarket & Grocery store",
    "Supermarket/Grocery Store": "Supermarket & Grocery store",
    "Supermarket / Grocery Store": "Supermarket & Grocery store",
    "SPS-District K-12": "K-12 School",
    "Senior Care Community": "Small & Medium medical facility",  # Unsure
    "Laboratory": "Small & Medium medical facility",  # Unsure
    "Medical Office": "Small & Medium medical facility",  # Unsure
    "Small- and Mid-Sized Office": "Small & Mid-Sized Office",
    "Non-Refrigerated Warehouse": "Warehouse",
    "Residence Hall/Dormitory": "Residence Hall",
    "University": "College/University",
    "Retail Store": "Supermarket & Grocery store",
    "Self-Storage Facility": "Warehouse",  # Unsure
}


In [ ]:
for key in correction_dict.keys():
    df_seattle["PrimaryPropertyType"].replace(to_replace=key, value=correction_dict[key], inplace=True)

print(df_seattle["PrimaryPropertyType"].value_counts())



In [ ]:
select_six = [
    "Small & Mid-Sized Office", "Large Office", "Warehouse", "Supermarket & Grocery store",
    "K-12 School", "Small & Medium medical facility",
]


In [ ]:
# EUI / E* , color code property type, sample based on smallest group size

fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 16),
    dpi=pc_dpi,
)

data_ptype = df_seattle[df_seattle["PrimaryPropertyType"].isin(select_six)]

ax1 = sns.boxplot(
    x=data_ptype["PrimaryPropertyType"],
    y=data_ptype["SourceEUI(kWh/m2)"],
    data=data_ptype
)

###
# Titles/Lables
ax1.set_xticklabels(
    labels=select_six,
    rotation=45
    )
ax1.set_ylim(0, 2500)  # few outliers above
fig.suptitle("Efficacité d'utilisation energetique en fonction de l'utilisation principale des principaux groupes de batiments")

#
###

plt.tight_layout()
plt.show()


#### Observations :
- Vérifier plus en detail la categorie K-12 peut se reveler interessant, les caracteristiques du groupe ressemble a celles d'une centrale a charbon
- Small / Mid sized office EUI < Large Office EUI : Influence de la taille globale sur l'efficacite energetique ?

In [ ]:
fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 16),
    dpi=pc_dpi,
)

ax1 = sns.scatterplot(
    x="PropertyArea(SquareMetre)Total",
    y="SiteEnergyUse(kWh)",
    hue="PrimaryPropertyType",
    data=data_ptype
)

###
# Titles/Lables
fig.suptitle("Utilisation d'energie en fonction de la taille des batiments, nuancés par leur utilisation principale")
#
###

plt.tight_layout()
plt.show()


In [ ]:
# Zooming on y=0.5 kWh/m2, x= 25km2

fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 10),
    dpi=pc_dpi,
)

ax1 = sns.scatterplot(
    x="PropertyArea(SquareMetre)Total",
    y="SiteEnergyUse(kWh)",
    hue="PrimaryPropertyType",
    data=data_ptype
)


ax1.set_ylim(-100, 15000000)
ax1.set_xlim(0, 25000)

###
# Titles/Lables
fig.suptitle("Utilisation d'energie en fonction de la taille des batiments, nuancés par leur utilisation principale, zoom sur y=0.5 kWh/m2, x= 25km2")
#
###

plt.tight_layout()
plt.show()

# Apply cleaning via IQR


#### Observations :
- Tres nombreux outliers, utilisation de la methode interquartile pour eliminer les valeurs aberrantes : 
  - Subsets : PrimaryPropertyType
  - Variables concernées : EUI, Size, Electricity